# Hackathon : Reinforcement Learning for Drone Navigation

- Team name: Olympus Coders
- Team members names: Loan SAMAI, Farès BENAGGOUNE, Marc Estassy BATABA

## 0. Prerequisites

In [33]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
%load_ext autoreload
%autoreload 2

import env
import agent
import reward
import agent
import simulate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Training

In [ ]:
# On peut passer un seul fichier de configuration ou une liste de plusieurs configs
# Exemple avec une seule config : trained_agent, all_rewards = simulate.multi_config_train(["train_configs/config_1.json"])
# Exemple avec plusieurs configs : trained_agent, all_rewards = simulate.multi_config_train(["train_configs/config_1.json", "train_configs/config_3.json", "train_configs/config_7.json"])

trained_agent, all_rewards = simulate.multi_config_train([
    "train_configs/config_small.json",
    "train_configs/config_medium.json",  # Commence par les environnements simples
    "train_configs/config_hybride.json", # Progresse vers les hybrides
    "train_configs/config_hard.json",    # Puis les difficiles
    "config.json",                       # Inclut aussi la config par défaut pour compléter l'apprentissage
], max_total_episodes=3000, checkpoint_path="multi_config_checkpoint.pth", save_interval=100)



🚀 Début de l'entraînement multi-configurations 🚀

Fichier de checkpoint non trouvé: multi_config_checkpoint.pth
Épisode 1/3000, Reward: -4208.21
Épisode 2/3000, Reward: -800.44
Épisode 3/3000, Reward: -11901.36
Épisode 4/3000, Reward: -6703.02
Épisode 5/3000, Reward: -7421.37
Épisode 6/3000, Reward: -11600.53
Épisode 7/3000, Reward: -7055.84
Épisode 8/3000, Reward: -13550.67
Épisode 9/3000, Reward: -6537.49
Épisode 10/3000, Reward: -18456.38
Épisode 11/3000, Reward: -144.89
Épisode 12/3000, Reward: -14446.20
Épisode 13/3000, Reward: -3106.30
Épisode 14/3000, Reward: -531.31
Épisode 15/3000, Reward: -75.42
Épisode 16/3000, Reward: -1837.37
Épisode 17/3000, Reward: -19445.89
Épisode 18/3000, Reward: -497.49
Épisode 19/3000, Reward: -15764.71
Épisode 20/3000, Reward: -25907.97
Épisode 21/3000, Reward: -2584.17
Épisode 22/3000, Reward: -97.26
Épisode 23/3000, Reward: -39334.60
Épisode 24/3000, Reward: -23425.53
Épisode 25/3000, Reward: -7402.21
Épisode 26/3000, Reward: -2993.46
Épisode 27

In [ ]:
# Plot the cumulated rewards per episode
simulate.plot_cumulated_rewards(all_rewards)

## 2. Evaluation

In [22]:
eval_config_paths = [f"./eval_configs/config_{i}.json" for i in range(1, 11)]

In [ ]:
trained_agent = agent.MyAgent(...)
trained_agent.load_checkpoint("multi_config_checkpoint_best.pth")  # le meilleur
all_results = simulate.evaluate(eval_config_paths, trained_agent, use_best_checkpoint=False)

In [ ]:
display(all_results)

In [ ]:
# Calculate averages for each configuration
averages = all_results.groupby('config_path').mean().reset_index().drop(columns=['episode'])
averages = averages.rename(columns={
    'steps': 'avg_steps',
    'reward': 'avg_reward',
    'evacuated': 'avg_evacuated',
    'deactivated': 'avg_deactivated'})

display(averages)
averages.to_csv('averages.csv', index=False)